In [1]:
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/POS_SCORE.csv')
df.head()

,review_Id,review,score,thumbs_Up_Count,review_App_Version,review_time,label,tokens,pos_tags,scores,compound,modified_scores
0,5854e2fd-b0af-44b5-9458-3efeb2369631,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,"['not', 'smoothest', 'easiest', 'apps', 'navig...","[('not', 'RB'), ('smoothest', 'JJS'), ('easies...","{'neg': 0.066, 'neu': 0.583, 'pos': 0.351, 'co...",0.9643,0.057111
1,2bbc0faa-43a2-4255-8035-23772541cb14,"the app itself has good features, eg let's you...",3,102,2.88.2,2024-10-02 09:35:00,Neutral,"['app', 'ha', 'good', 'feature', 'e', 'Eg', 'l...","[('app', 'JJ'), ('ha', 'NN'), ('good', 'JJ'), ...","{'neg': 0.0, 'neu': 0.794, 'pos': 0.206, 'comp...",0.8858,0.040849
2,204edfd3-a877-4d32-be56-02c1e27abc44,"an easy app to use. easy to navigate, fast at ...",5,84,2.89.0,2024-02-20 11:33:00,Positive,"['easy', 'app', 'use', '.', 'easy', 'navigate'...","[('easy', 'JJ'), ('app', 'NN'), ('use', 'NN'),...","{'neg': 0.078, 'neu': 0.68, 'pos': 0.242, 'com...",0.8799,0.030426
3,1181eb84-aa16-48ea-b455-623e05635048,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,"['pointless', 'app', '.', 'i', 'can', 'see', '...","[('pointless', 'NN'), ('app', 'NN'), ('.', '.'...","{'neg': 0.144, 'neu': 0.784, 'pos': 0.071, 'co...",-0.5883,-0.013219
4,a074686c-acef-41e8-9e73-c0d9cd5dd04b,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,"['update', '.', 'issue', 'WA', 'resolved', 'qu...","[('update', 'NN'), ('.', '.'), ('issue', 'NN')...","{'neg': 0.193, 'neu': 0.542, 'pos': 0.265, 'co...",0.6240,0.034635


In [4]:
columns_to_drop = ['scores', 'compound', 'modified_scores']
df.drop(columns=columns_to_drop, inplace=True)
df

,review_Id,review,score,thumbs_Up_Count,review_App_Version,review_time,label,tokens,pos_tags
0,5854e2fd-b0af-44b5-9458-3efeb2369631,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,"['not', 'smoothest', 'easiest', 'apps', 'navig...","[('not', 'RB'), ('smoothest', 'JJS'), ('easies..."
1,2bbc0faa-43a2-4255-8035-23772541cb14,"the app itself has good features, eg let's you...",3,102,2.88.2,2024-10-02 09:35:00,Neutral,"['app', 'ha', 'good', 'feature', 'e', 'Eg', 'l...","[('app', 'JJ'), ('ha', 'NN'), ('good', 'JJ'), ..."
2,204edfd3-a877-4d32-be56-02c1e27abc44,"an easy app to use. easy to navigate, fast at ...",5,84,2.89.0,2024-02-20 11:33:00,Positive,"['easy', 'app', 'use', '.', 'easy', 'navigate'...","[('easy', 'JJ'), ('app', 'NN'), ('use', 'NN'),..."
3,1181eb84-aa16-48ea-b455-623e05635048,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,"['pointless', 'app', '.', 'i', 'can', 'see', '...","[('pointless', 'NN'), ('app', 'NN'), ('.', '.'..."
4,a074686c-acef-41e8-9e73-c0d9cd5dd04b,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,"['update', '.', 'issue', 'WA', 'resolved', 'qu...","[('update', 'NN'), ('.', '.'), ('issue', 'NN')..."
...,...,...,...,...,...,...,...,...,...
194,dc33557a-e667-404d-8954-42caa8239674,i'm grateful for the barclays app.its helped m...,5,0,2.88.2,2024-02-02 02:47:00,Positive,"['i', 'm', 'grateful', 'barclays', 'appoints',...","[('i', 'JJ'), ('m', 'VBP'), ('grateful', 'JJ')..."
195,700256ec-0e22-4120-9124-7a1d05d81a16,"overall, it's a good app. however, you need to...",4,0,2.88.2,2024-03-02 07:45:00,Positive,"['overall', 'e', 's', 'good', 'app', '.', 'how...","[('overall', 'JJ'), ('e', 'NN'), ('s', 'NN'), ..."
196,9706b3f2-345c-4186-a11e-2e3930c032a1,technical team takes up to five days to resolv...,2,1,2.88.2,2024-01-19 18:46:00,Neutral,"['technical', 'team', 'take', 'five', 'day', '...","[('technical', 'JJ'), ('team', 'NN'), ('take',..."
197,0cb78aa7-2f23-48c5-ad80-7b2fed88e38f,"update 2, still as crass as before with inexpl...",1,0,2.88.2,2024-01-02 08:21:00,Negative,"['update', '2', 'e', 'still', 'crass', 'before...","[('update', 'JJ'), ('2', 'CD'), ('e', 'NN'), (..."


In [5]:
# Parse the 'tokens' column to convert string representations of lists into actual lists
df['tokens'] = df['tokens'].apply(literal_eval)


In [6]:
# Train a Word2Vec model on the tokens
# Set minimum word count to 2 and vector size to 100 for the purpose of this demonstration
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=2, workers=4)

In [7]:
# Function to vectorize a sentence by averaging all word vectors in the sentence
def vectorize_sentence(tokens, model):
    vector = [model.wv[word] for word in tokens if word in model.wv]
    if len(vector) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(vector, axis=0)

In [8]:
df['token_vectors'] = df['tokens'].apply(lambda x: vectorize_sentence(x, word2vec_model))


In [9]:
# Display the first few vectors to verify
df['token_vectors'].head()

0    [-0.19719312, 0.3657256, -0.1189997, 0.073821,...
1    [-0.18274723, 0.33667773, -0.11073305, 0.06730...
2    [-0.19563282, 0.3601184, -0.11732106, 0.071292...
3    [-0.18884978, 0.34954417, -0.11485775, 0.06960...
4    [-0.18822053, 0.34687746, -0.114304036, 0.0697...
Name: token_vectors, dtype: object

In [13]:
# DataFrame df with 'review' as text data and 'label' as corresponding labels
X = np.array(df['token_vectors'].tolist())
y = df['label']

In [14]:
# Assume 'X' contains the tokenized reviews and 'y' contains the corresponding labels (positive, negative, neutral)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [15]:
#Create a Support Vector Machine classifier and Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [16]:
y_pred = svm_classifier.predict(X_test)

In [17]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.35
              precision    recall  f1-score   support

    Negative       0.35      1.00      0.52        21
     Neutral       0.00      0.00      0.00        25
    Positive       0.00      0.00      0.00        14

    accuracy                           0.35        60
   macro avg       0.12      0.33      0.17        60
weighted avg       0.12      0.35      0.18        60



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
#LOGISTIC REGRESSION MODEL
from sklearn.linear_model import LogisticRegression

In [19]:
# Create a Logistic Regression classifier and Train the LR classifier
logreg_classifier = LogisticRegression()
# Train the classifier
logreg_classifier.fit(X_train, y_train)

LogisticRegression()

In [20]:
y_pred = logreg_classifier.predict(X_test)

In [21]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3
Classification Report:
               precision    recall  f1-score   support

    Negative       0.30      0.67      0.42        21
     Neutral       0.29      0.16      0.21        25
    Positive       0.00      0.00      0.00        14

    accuracy                           0.30        60
   macro avg       0.20      0.28      0.21        60
weighted avg       0.23      0.30      0.23        60



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
#NAIVES BAYES MODEL
from sklearn.naive_bayes import MultinomialNB

In [25]:
# Create a Naives Bayes Model classifier and Train the NB classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

ValueError: Negative values in data passed to MultinomialNB (input X)

In [26]:
#NAIVES BAYES IS NOT DESIGNED FOR NEGATIVE VALUES

In [27]:
#RANDOM FOREST MODEL
from sklearn.ensemble import RandomForestClassifier

In [28]:
# Create a Random Forest classifier and Train the classifier
rf_classifier = RandomForestClassifier(n_estimators=2000, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000, random_state=42)

In [29]:
y_pred = rf_classifier.predict(X_test)

In [30]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.38333333333333336
Classification Report:
               precision    recall  f1-score   support

    Negative       0.30      0.38      0.33        21
     Neutral       0.53      0.40      0.45        25
    Positive       0.36      0.36      0.36        14

    accuracy                           0.38        60
   macro avg       0.39      0.38      0.38        60
weighted avg       0.41      0.38      0.39        60

